# 객체 추적 함수
- cv2.\__version\__ : 3.4.11
- conda --version : 22.9.0
- python --version : 3.9.13
- tracker
    - csrt : cv2.TrackerCSRT_create
    - kcf : cv2.TrackerKCF_create
    - boosting : cv2.TrackerBoosting_create
    - mil : cv2.TrackerMIL_create
    - tld : cv2.TrackerTLD_create
    - medianflow : cv2.TrackerMedianFlow_create
    - mosse : cv2.TrackerMOSSE_create
    정확도가 좋으면 속도가 느립니다.

In [58]:
import cv2
import os

# 이미지 저장 시 자동으로 번호 메기기
def count_img(video_path):
    
    num = 1
    
    try : 
        video_name = video_path.split('/')[-1].split('.')[0]
        path_file_list = os.listdir(os.getcwd())
    
        for file in path_file_list:
            if video_name in file:
                num += 1

        return (video_name, num)
    except :
        print('이미지와 동영상의 위치를 확인해주세요')
        return

# 객체 추적 알고리즘
def run_object_tracking(video_path):
    cap = cv2.VideoCapture(video_path)
    
    # 영상객체를 정상적으로 불러온 경우
    if cap.isOpened():
        
        fps = cap.get(cv2.CAP_PROP_FPS)
        delay = int(1000/fps)
        
        tracker = cv2.TrackerCSRT_create()
        video_name, num = count_img(video_path)

        while True :
            ret, frame = cap.read()
            # 프레임을 읽지 못한 경우
            if not ret:
                print("Can't read video file")
                break

            img_draw = frame.copy()

            # 트래커 생성이 안된 경우
            if tracker is None:
                print(tracker)
                cv2.putText(img_draw, "No Tracker!!!", (30,30),\
                            cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,0,255), 2, cv2.LINE_AA)

            # 새로운 프레임에서 추적 위치 찾기
            else :
                ok, bbox = tracker.update(frame)
                (x,y,w,h) = bbox
                # 추적 성공
                if ok:
                    cv2.rectangle(img_draw, (int(x), int(y)), (int(x+w),int(y+h)), \
                                            (0,255,0), 2, 1)
                # 추적 실패
                else :
                    cv2.putText(img_draw, "Tracking fail", (30,30), \
                               cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,255,0),2,cv2.LINE_AA)

            cv2.imshow('tracking', img_draw)
            key = cv2.waitKey(delay) & 0xff
            if key == ord(' '):
                # 초기 객체 위치 설정
                roi = cv2.selectROI('tracking', frame, False)

                # roi 영역을 선택하지 않았을 경우
                if roi == (0,0,0,0) :
                    cv2.imshow('tracking', frame)
                elif any(roi) :
                    tracker = cv2.TrackerCSRT_create()
                    tracker.init(frame, roi)
            
            elif key == 27: #ESC
                break
            
            elif key == 49: # 1
                cv2.imwrite(f"./{video_name}{num}.jpg", frame)
                num += 1
    else:
        print("Could not open video")
    cap.release()
    cv2.destroyAllWindows()
    
video_path = './data/load.mp4'
run_object_tracking(video_path)